## Juego del Prisionero con retraso en la difusión de la información

En este trabajo práctico vamos a analizar una variante del [Dilema del Prisionero](https://es.wikipedia.org/wiki/Dilema_del_prisionero).

[Existen múltiples modificaciones al juego](http://systems-sciences.uni-graz.at/etextbook/gametheory/prisonersdilemma.html). Una de las ramas estudiadas es la del juego iterativo: se juegan sucesivas veces, y en cada partida los agentes tienen cierta memoria de lo que hizo el otro en el pasado. Existen también estudios hechos con múltiples agentes, varias estrategias en el tiempo y distintas topologías (quién juega contra quién).

En nuestra variante la grilla representa una prisión y los prisioneros se mueven de forma aleatoria por ella, y al encontrarse con otro prisionero deciden o no traicionarlo.
Al ocurrir una traición, se aumenta la condena del traicionado en una cierta cantidad de tiempo, pero la víctima se enterará de que fue traicionado un determinado tiempo después.

La mejor solución es Quid-Pro-Quo, ya que fue estudiado que ésta reduce la dispersión de los tiempos de condena, pero esta se basa en la inmediatez con la que un prisionero conoce que otro lo delató o traicionó. En una situación de esparcimiento de información falsa o no comprobable (como esta)
con objetivos de manipulación, para conseguir una reducción de la condena un alcaide es informado de que alguien traiciona a alguien y éste decide que hace con la información. Esto es lo que deseamos modelar.

### Modelo conceptual

En general nuestro modelo se explica por:

- un tamaño de la grilla
- una cantidad de prisioneros inicial (`N`) con sus respectivas condenas iniciales
- encuentros entre prisioneros que con probabilidad `1/p` desata una traición informada al oponente con retraso `m,..,M` iteraciones (equiprobable)
- encuentros entre prisioneros que ya saben que fueron buchoneados por su oponente en algún instante anterior y entonces siempre traicionan (Quid-Pro-Quo) con retraso de información de `m,..,M` iteraciones (equiprobable)
- el paso del tiempo
- cuánto tiempo se agrega a la condena cuando se es traicionado (`c`)

Si bien sobre esta variante introdujimos algunas particularidades:

- si un prisionero todavía no sabe que fue buchoneado por otro que se vuelve a encontrar, entonces la condena se aumenta pero se toma el mínimo retraso de información, entre la nueva traición y la anterior
- las condenas nunca se reducen
- no se distingue entre traición-traición y traición-no traición (a diferencia del Dilema original cuya condena agregada es menor en el primer caso)
- si la condena es 0, el prisionero inmediatamente sale de la prisión

Con el objetivo de entender el modelo, experimentaremos con distintos valores de los parámetros:
- `c`: agregado leve (1 turno), moderado (10 turnos), grave (50 turnos)
- `p`: probabilidad baja (0.2), media (0.5), alta (0.8)
- `m` y `M`: retraso rápido (0 a 5 turnos), variado (0 a 10 turnos), medio (5 a 10 turnos), largo (10 a 20 turnos)

Los prisioneros inician con una condena aleatoria (distribución normal, con media 50 turnos y desvio estándar 10 turnos), en posiciones iniciales seleccionadas de forma aleatoria. Para cara experimento, corremos 15 repeticiones con condenas y posiciones distintas.

El movimiento se realiza de forma aleatoria: en cada turno, un prisionero se mueve a la posición a la que está mirando (si puede). Al moverse, elige una nueva dirección a la que mirar, en la vecindad de von Neumann. Para evitar colisiones (dos prisioneros moviéndose a la misma celda), el autómata celular utiliza una vecindad de von Neumann extendida.

Definimos un conflicto cuando dos prisioneros se encuentran en celdas vecinas (en la vecindad de von Neumann) y ambos se miran respectivamente. Cada vez que sucede esto, ambos prisioneros pueden ser buchoneados por el otro, siguiendo las reglas nombradas anteriormente (y formalizadas en la próxima sección).

### Reglas Cell-DEVS

Para esto usamos CD++ y un archivo ``.ma`` generado de forma dinámica, pero cuyas bases son las reglas de:

- _Movimiento_: mueven a un prisionero en la dirección a la que está mirando si el casillero está libre, además hacen que pase el tiempo.
- _Eliminación_: remueven a un prisionero que se movió de lugar del casillero anterior.
- _Buchoneo_: determinan si mi oponente me buchonea y cuanto tiempo pasa hasta que me entere. Se usa 0 como código de que fuí traicionado, -1 para no haber sido traicionado y n > 0 como la cantidad de iteraciones que faltan para saber que me traicionaron.

Se codifica una tupla en la que:
- Lugar 0-ésimo: identificador del prisionero
- Lugar 1-ésimo: dirección (de mirada)
- Lugar 2-ésimo: duración de la condena
- Lugar i+3-ésimo: 0 si el prisionero i me traicionó y lo sé, n > 0 si faltan n iteraciones para saber que el prisionero i me traicinó y -1 si nunca fue traicionado por éste.

NOTA: las reglas y sus condiciones se encuentras comentadas para mayor claridad.

In [ ]:
%%bash

tail -n +20 model/prisioneros.ma.j2 | head -n 55

In [ ]:
%%bash

# Mostramos la regla de buchoneo solamente para uno de los 4 casos.
# los otros casos son similares

tail -n +80 model/prisioneros.ma.j2 | head -n 30

In [ ]:
%%bash

tail -n +204 model/prisioneros.ma.j2 | head -n 30

In [ ]:
%%bash

tail -n +234 model/prisioneros.ma.j2

### Experimentos

In [ ]:
from log_loader import load_log
from plots import agents_count_stats, plot_uncertainty, total_life_times, plot_time_histogram
from plots import search_experiment_results, get_conflicts, proba_text, log_index, indices
# log_index es un diccionario de id de experimento a (c, (m, M), p)
from model.experiment_utils import sentences, delays, betrayal_probas
import matplotlib.pyplot as plt

In [ ]:
log_dfs = search_experiment_results(100, (0, 5), proba_text(20))
stats = agents_count_stats(log_dfs)
plot_uncertainty(stats, 'Experimento 100 de condena agregada, 0 a 5 turnos para avisar buchoneo, 20% probabilidad de buchoneo')

In [ ]:
# La sumatoria concatena listas
life_times = sum([total_life_times(log_df) for log_df in log_dfs], [])

# Histograma del tiempo de condena total de los agentes con distintas corridas aleatorizadas
plot_time_histogram(life_times, 'Tiempo de condena total')

### Tareas

#### TOBI

- [X] Grafico y correlacion entre variables y buchoneos (regresion lineal) -> siendo tan pocas categorias no se si tiene sentido regresion lineal
- [X] Buchoneos por configuracion (heatmap de a dos variables)
- [ ] Corrleacion entre veces que buchoneaste y veces que fuiste buchoneado (recta)

##### VALEN

- [ ] Buchoneos en funcion del tiempo (comparar barplots)
- [ ] Comparar buchoneos segun retardo alto con retardo bajo segun el tiempo (en forma de rectas)
- [ ] Para ver si hay distintos momentos (pocos buchonean vs todos buchonean xq saben que fueron delatados) segun el retardo

#### Cantidad de buchoneos en función del resto de variables

Veremos como varías la cantidad de traiciones entre agentes dependiendo de las variables experimentales. A priori, lo que esperamos es:

- Que a mayor condena agregada por buchoneo, mayor sea la condena total de los agentes, por lo que tendrán más tiempo para buchonear a otros.
- A mayor tiempo de retraso, menor cantidad de buchoneos, ya que los agentes seguirán las dinámica aleatoria de buchoneos por más tiempo
- A mayor probabilidad, habrá obviamente más buchoneos, aunque nos parece interesante ver cuánto cambia

Realizaremos mapas de color por cada par de variables experimentales, para ver si emergen sinergias entre las variables.

In [ ]:
!make conflicts.csv

In [ ]:
import pandas as pd

def parse_tuple(value):
    return tuple(int(float(num)) for num in value.replace('(', '').replace(')', '').split(', '))

num_conflicts = pd.read_csv('conflicts.csv', converters={'d': parse_tuple})
num_conflicts

In [ ]:
def plot_many_histograms(xs, labels, bins_size):
    max_val = pd.concat(xs).max() + bins_size
    fig, axs = plt.subplots(len(xs), sharex=True, sharey=True, figsize=(9, 4*len(xs)))
    for x,l,ax in zip(xs, labels,axs):
        ax.hist(x, bins=list(range(0, max_val, bins_size)), label=l)
        ax.axvline(x.mean(), color='red', label=f'Media = {round(x.mean(), 2)}')
        ax.axvline(x.median(), color='green', label=f'Mediana = {round(x.median(), 2)}')
        ax.legend()
        ax.grid()
    plt.show()

In [ ]:
plot_many_histograms([num_conflicts[num_conflicts['c'] == c]['num'] for c in sentences], sentences, 50)

In [ ]:
def proba_from_text(t):
    d = {"randInt(4) = 0": "20%", "randInt(1) = 0": "50%", "randInt(4) != 0": "80%"}
    assert t in d.keys()
    return d[t]
plot_many_histograms([num_conflicts[num_conflicts['p'] == p]['num'] for p in betrayal_probas],
                     map(proba_from_text, betrayal_probas), 50)

In [ ]:
plot_many_histograms([num_conflicts[num_conflicts['d'] == d]['num'] for d in delays], map(str,delays), 50)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_heatmap(rows, cols, grouped_dataframe):
    plt.figure()
    plt.imshow([[grouped_dataframe[r,c] for c in cols] for r in rows])
    plt.xticks(np.arange(len(cols)), labels=cols)
    plt.yticks(np.arange(len(rows)), labels=rows)
    # Loop over data dimensions and create text annotations.
    for i, r in enumerate(rows):
        for j, c in enumerate(cols):
            plt.text(j, i, grouped_dataframe[r, c], ha="center", va="center", color="w")
    plt.show()

In [ ]:
plot_heatmap(sentences, delays, num_conflicts.groupby(['c','d'])['num'].sum())

In [ ]:
plot_heatmap(sentences, betrayal_probas, num_conflicts.groupby(['c','p'])['num'].sum())

In [ ]:
plot_heatmap(betrayal_probas, delays, num_conflicts.groupby(['p','d'])['num'].sum())

In [ ]:
!make conflicts_compare.csv

In [ ]:
num_conflicts_agents = pd.read_csv('conflicts_compare.csv', converters={'d': parse_tuple})
num_conflicts_agents

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(num_conflicts_agents['buchoneado'], num_conflicts_agents['buchonea'])
plt.xlabel('Veces que el agente fue buchoneado')
plt.xlabel('Veces que el agente buchoneó')
plt.show()

In [ ]:
def many_scatters(xs, ys, labels):
    fig, axs = plt.subplots(1, len(xs), sharex=True, sharey=True, figsize=(5*len(xs), 5))
    for x,y,l,ax in zip(xs,ys,labels,axs):
        ax.scatter(x, y, label=l)
        ax.set_xlabel('Veces que el agente fue buchoneado')
        ax.set_xlabel('Veces que el agente buchoneó')
        ax.legend()
    plt.show()

In [ ]:
many_scatters([num_conflicts_agents[num_conflicts_agents['c'] == c]['buchoneado'] for c in sentences],
              [num_conflicts_agents[num_conflicts_agents['c'] == c]['buchonea'] for c in sentences],
              sentences)

In [ ]:
many_scatters([num_conflicts_agents[num_conflicts_agents['d'] == d]['buchoneado'] for d in delays],
              [num_conflicts_agents[num_conflicts_agents['d'] == d]['buchonea'] for d in delays],
              map(str, delays))

In [ ]:
many_scatters([num_conflicts_agents[num_conflicts_agents['p'] == p]['buchoneado'] for p in betrayal_probas],
              [num_conflicts_agents[num_conflicts_agents['p'] == p]['buchonea'] for p in betrayal_probas],
              map(proba_from_text, betrayal_probas))